<a href="https://colab.research.google.com/github/jmorala/TFMDS/blob/main/cuadernos/01_exploracion_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicializar directorios
Clonar repositorio github
Posicionarse en el directorio raíz

In [1]:
# Reemplaza 'tu_usuario' con tu nombre de usuario de GitHub
# Este comando descarga el repositorio entero a una carpeta llamada 'TFMDS' en Colab.
!git clone https://github.com/jmorala/TFMDS.git

Cloning into 'TFMDS'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 36 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 4.07 MiB | 3.37 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
import os

# Entra en la carpeta del proyecto
os.chdir('TFMDS')

# OPCIONAL: Para verificar que estás en la ruta correcta y ver las carpetas
print("Directorio de trabajo actual:", os.getcwd())
print("Contenido del directorio:")
!ls -l

Directorio de trabajo actual: /content/TFMDS
Contenido del directorio:
total 16
drwxr-xr-x 2 root root 4096 Nov  4 15:27 cuadernos
drwxr-xr-x 2 root root 4096 Nov  4 15:27 datos
drwxr-xr-x 2 root root 4096 Nov  4 15:27 documentacion
-rw-r--r-- 1 root root   63 Nov  4 15:27 README.md


# Revisar datos

In [5]:
import pandas as pd

# Define la ruta relativa de tu archivo CSV (ajusta el nombre del archivo)
RUTA_DATOS = 'datos/Ventas.csv'

# Carga el archivo en un DataFrame de Pandas
df = pd.read_csv(RUTA_DATOS, sep=';',
    parse_dates=['idSecuencia'],
    date_format='%Y%m%d')

# Muestra las primeras filas y la información de las columnas para iniciar la exploración
print("Primeras 5 filas del DataFrame:")
print(df.head())

print("\nInformación de las columnas y tipos de datos:")
df.info()

Primeras 5 filas del DataFrame:
   producto idSecuencia  udsVenta
0         1  2022-11-05        40
1         1  2022-11-06         0
2         1  2022-11-07        12
3         1  2022-11-08        28
4         1  2022-11-09        14

Información de las columnas y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654408 entries, 0 to 654407
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   producto     654408 non-null  int64         
 1   idSecuencia  654408 non-null  datetime64[ns]
 2   udsVenta     654408 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 15.0 MB


In [14]:
prod_vc = df.producto.value_counts ()
print(prod_vc)
print(f'lecturas diferentes: {prod_vc.nunique()}')

producto
1000    732
1       732
2       732
3       732
4       732
       ... 
20      732
19      732
18      732
17      732
16      732
Name: count, Length: 894, dtype: int64
lecturas diferentes: 1


El número de productos es 894 y tienen 732 lecturas cada uno